## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-11-16-12-57 +0000,nypost,LA tax hike that will raise price of groceries...,https://nypost.com/2026/02/11/us-news/la-sales...
1,2026-02-11-16-10-42 +0000,nypost,Trump praises NYS gov candidate Bruce Blakeman...,https://nypost.com/2026/02/11/us-news/trump-pr...
2,2026-02-11-16-10-17 +0000,nyt,House Is Set to Vote on Canceling Trump’s Cana...,https://www.nytimes.com/2026/02/11/us/politics...
3,2026-02-11-16-08-29 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/02/11/us/faa...
4,2026-02-11-16-08-10 +0000,bbc,Russian strike kills 3 Ukrainian toddlers and ...,https://www.bbc.com/news/articles/crm820dp0yro...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_3060/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,56
26,canada,19
446,epstein,18
120,new,17
155,guthrie,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
38,2026-02-11-15-19-00 +0000,wsj,House lawmakers rejected an attempt by Speaker...,https://www.wsj.com/politics/policy/house-reje...,169
113,2026-02-11-11-57-00 +0000,wsj,Commerce Secretary Howard Lutnick became the f...,https://www.wsj.com/politics/policy/lutnicks-f...,116
205,2026-02-11-03-51-40 +0000,bbc,Under-fire Trump commerce secretary confirms h...,https://www.bbc.com/news/articles/cwyx9vq0ne0o...,112
237,2026-02-11-01-14-19 +0000,latimes,L.A. County officials push new sales tax to of...,https://www.latimes.com/california/story/2026-...,107
253,2026-02-10-23-53-00 +0000,wsj,Canada’s Carney Tries to Resolve Trump Demands...,https://www.wsj.com/world/americas/carney-look...,105


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
38,169,2026-02-11-15-19-00 +0000,wsj,House lawmakers rejected an attempt by Speaker...,https://www.wsj.com/politics/policy/house-reje...
218,80,2026-02-11-02-45-29 +0000,nyt,New Photos Show Masked Person on Nancy Guthrie...,https://www.nytimes.com/2026/02/10/us/nancy-gu...
176,67,2026-02-11-09-03-03 +0000,nypost,‘Gunperson’ in a ‘dress’ behind Canada’s deadl...,https://nypost.com/2026/02/11/world-news/tumbl...
113,60,2026-02-11-11-57-00 +0000,wsj,Commerce Secretary Howard Lutnick became the f...,https://www.wsj.com/politics/policy/lutnicks-f...
0,43,2026-02-11-16-12-57 +0000,nypost,LA tax hike that will raise price of groceries...,https://nypost.com/2026/02/11/us-news/la-sales...
182,43,2026-02-11-08-37-08 +0000,wsj,Stock News Today: Dow Slips; Jobs Report Weake...,https://www.wsj.com/livecoverage/jobs-report-u...
131,41,2026-02-11-11-09-51 +0000,nypost,Ex-Prince Andrew’s gushing emails to Epstein b...,https://nypost.com/2026/02/11/world-news/ex-pr...
363,40,2026-02-10-18-38-47 +0000,nypost,Indicted Dem asks ICE acting director Todd Lyo...,https://nypost.com/2026/02/10/us-news/indicted...
32,36,2026-02-11-15-28-50 +0000,nypost,Brother of dead NFL war hero Pat Tillman admit...,https://nypost.com/2026/02/11/us-news/brother-...
60,35,2026-02-11-14-35-27 +0000,nypost,Warner Bros. Discovery faces activist investor...,https://nypost.com/2026/02/11/business/wbd-fac...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
